In [ ]:
#!/usr/bin/env python
# coding: utf-8

import os
import serial
import threading
import datetime
import gspread
from google.oauth2.service_account import Credentials
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import queue
import csv
import cv2
import time

# Column headers for the Google Spreadsheet
COLUMN_HEADERS = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Google Sheets scope and client initialization
SCOPE = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

# Global stop event for threads
stop_event = threading.Event()

# Splash screen class
class SplashScreen:
    def __init__(self, root, duration=3000):
        self.root = root
        self.root.overrideredirect(True)
        self.root.attributes("-alpha", 0.0)

        # Resize splash screen for 7-inch screen (800x480 resolution)
        width, height = 800, 480
        x = (self.root.winfo_screenwidth() - width) // 2
        y = (self.root.winfo_screenheight() - height) // 2
        self.root.geometry(f"{width}x{height}+{x}+{y}")
        self.root.configure(bg="black")

        self.label = tk.Label(self.root, text="McCutcheonlab Technologies", font=("Cascadia Code", 28, "bold"), bg="black", fg="violet")
        self.label.pack(expand=True)
        self.duration = duration
        self.alpha = 0.0  # Initialize alpha value for fade effects
        self.fade_in()

    def fade_in(self):
        if self.alpha < 1.0:
            self.alpha += 0.05
            self.root.attributes("-alpha", self.alpha)
            self.root.after(50, self.fade_in)
        else:
            self.root.after(2000, self.fade_out)  # Stay for 2 seconds before fading out

    def fade_out(self):
        if self.alpha > 0.0:
            self.alpha -= 0.05
            self.root.attributes("-alpha", self.alpha)
            self.root.after(50, self.fade_out)
        else:
            self.root.destroy()

# Main GUI Class
class FED3MonitorApp:

    def __init__(self, root):
        self.root = root
        self.root.title("Realtime FED Monitor")
        self.root.geometry("800x480")  # Set for 7-inch touchscreen resolution

        # Variables for the GUI
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()
        self.json_path = tk.StringVar()
        self.spreadsheet_id = tk.StringVar()
        self.save_path = ""
        self.data_queue = queue.Queue()
        self.threads = []
        self.port_widgets = {}
        self.recording_circle = None
        self.recording_label = None
        self.data_to_save = {}

        self.gspread_client = None
        self.cameras = {}
        self.serial_ports = {}
        self.device_mapping = {}  # Mapping of serial ports to camera indices

        # Per-device recording states and timing
        self.recording_states = {}    # To track recording status per serial port
        self.last_event_times = {}    # To track last 'Pellet' event time per serial port
        self.recording_locks = {}     # To synchronize access per serial port

        self.setup_gui()
        self.setup_ports()  # Initialize serial ports and cameras

    def setup_gui(self):
        # Resize GUI elements to fit 7-inch screen
        self.root.grid_columnconfigure((0, 1, 2, 3), weight=1)
        
        # Experimenter name input
        tk.Label(self.root, text="Your Name:", font=("Cascadia Code", 8, "bold")).grid(column=0, row=0, sticky=tk.E, padx=2, pady=2)
        self.experimenter_entry = ttk.Entry(self.root, textvariable=self.experimenter_name, width=12)
        self.experimenter_entry.grid(column=1, row=0, sticky=tk.W, padx=2, pady=5)

        # Experiment name input
        tk.Label(self.root, text="Experiment Name:", font=("Cascadia Code", 8, "bold")).grid(column=2, row=0, sticky=tk.E, padx=2, pady=2)
        self.experiment_entry = ttk.Entry(self.root, textvariable=self.experiment_name, width=15)
        self.experiment_entry.grid(column=3, row=0, sticky=tk.E, padx=2, pady=2)

        # JSON file path
        tk.Label(self.root, text="Google API JSON File:", font=("Cascadia Code", 8, "bold")).grid(column=0, row=1, sticky=tk.E, padx=2, pady=2)
        self.json_entry = ttk.Entry(self.root, textvariable=self.json_path, width=25)
        self.json_entry.grid(column=1, row=1, columnspan=2, sticky=tk.W, padx=2, pady=2)
        self.browse_json_button = tk.Button(self.root, text="Browse", command=self.browse_json)
        self.browse_json_button.grid(column=3, row=1, padx=2, pady=2)

        # Google Spreadsheet ID
        tk.Label(self.root, text="Google Spreadsheet ID:", font=("Cascadia Code", 8, "bold")).grid(column=0, row=2, sticky=tk.E, padx=2, pady=2)
        self.spreadsheet_entry = ttk.Entry(self.root, textvariable=self.spreadsheet_id, width=25)
        self.spreadsheet_entry.grid(column=1, row=2, columnspan=2, sticky=tk.W, padx=2, pady=2)

        # Start button
        self.start_button = tk.Button(self.root, text="START", font=("Cascadia Code", 10, "bold"), bg="green", fg="white", command=self.start_logging)
        self.start_button.grid(column=0, row=3, padx=5, pady=5, sticky=tk.W)

        # Stop button
        self.stop_button = tk.Button(self.root, text="STOP(SAVE & QUIT)", font=("Cascadia Code", 10, "bold"), bg="red", fg="white", command=self.stop_logging)
        self.stop_button.grid(column=1, row=3, padx=5, pady=5, sticky=tk.W)

        # Browse button for selecting data folder path
        self.browse_button = tk.Button(self.root, text="Browse Data Folder", font=("Cascadia Code", 10, "bold"), command=self.browse_folder, bg="gold", fg="blue")
        self.browse_button.grid(column=2, row=3, padx=5, pady=5, sticky=tk.W)

        # Canvas for Recording Indicator
        self.canvas = tk.Canvas(self.root, width=100, height=100)
        self.canvas.grid(column=2, row=4, pady=5, sticky=tk.N)

    def browse_json(self):
        self.json_path.set(filedialog.askopenfilename(title="Select JSON File"))

    def browse_folder(self):
        self.save_path = filedialog.askdirectory(title="Select Folder to Save Data")

    def setup_ports(self):
        self.serial_ports = {}
        self.cameras = {}
        self.threads = []

        # Manually define your device mapping here
        # Adjust this mapping based on your actual setup
        # E.g., {'/dev/ttyACM0': 0, '/dev/ttyACM1': 1}
        self.device_mapping = {
            '/dev/ttyACM0': 0,  
            '/dev/ttyACM1': 1,
            # Add more mappings if you have more devices
        }

        # Prepare port widgets
        idx = 0
        for serial_port, camera_index in self.device_mapping.items():
            port_name = f"Port {idx + 1}"
            frame = ttk.LabelFrame(self.root, text=port_name)
            row_position = 4 + (idx // 2)
            column_position = idx % 2
            frame.grid(column=column_position, row=row_position, padx=5, pady=5, sticky=(tk.N, tk.S, tk.E, tk.W))
            status_label = ttk.Label(frame, text="Initializing...", font=("Cascadia Code", 8, "italic"), foreground="orange")
            status_label.grid(column=0, row=0, sticky=tk.W)
            text_widget = tk.Text(frame, width=33, height=5)
            text_widget.grid(column=0, row=1, sticky=(tk.N, tk.S, tk.E, tk.W))
            self.port_widgets[serial_port] = {'frame': frame, 'status_label': status_label, 'text_widget': text_widget}
            
            # Initialize serial port
            try:
                ser = serial.Serial(serial_port, 115200, timeout=1)
                self.serial_ports[serial_port] = ser
                status_label.config(text="Ready", foreground="green")
            except Exception as e:
                print(f"Serial port {serial_port} not connected: {e}")
                status_label.config(text="Serial Not Connected", foreground="red")
                idx +=1
                continue  # Skip to next if serial port is not connected

            # Initialize camera
            cam = cv2.VideoCapture(camera_index)
            if cam.isOpened():
                self.cameras[serial_port] = cam
                status_label.config(text="Ready (CAM Ready)", foreground="green")
            else:
                print(f"Camera index {camera_index} not connected or unavailable.")
                status_label.config(text="Ready (CAM Not Connected)", foreground="orange")
                cam.release()

            # Initialize recording state and lock
            self.recording_states[serial_port] = False
            self.last_event_times[serial_port] = None
            self.recording_locks[serial_port] = threading.Lock()

            idx +=1

    def start_logging(self):
        if not self.save_path:
            messagebox.showerror("Error", "Please select a folder to save data before starting.")
            return

        # Disable entry fields and normalize inputs
        self.experimenter_entry.config(state='disabled')
        self.experiment_entry.config(state='disabled')
        self.json_entry.config(state='disabled')
        self.spreadsheet_entry.config(state='disabled')
        
        self.experimenter_name.set(self.experimenter_name.get().strip().lower())
        self.experiment_name.set(self.experiment_name.get().strip().lower())
        self.json_path.set(self.json_path.get().strip())
        self.spreadsheet_id.set(self.spreadsheet_id.get().strip())

        try:
            creds = Credentials.from_service_account_file(self.json_path.get(), scopes=SCOPE)
            self.gspread_client = gspread.authorize(creds)
            print(f"Connected to Google Spreadsheet: {self.spreadsheet_id.get()}")
        except Exception as e:
            messagebox.showerror("Error", f"Error setting up Google Sheets API: {e}")
            return

        self.show_recording_indicator()
        self.start_data_collection()

    def show_recording_indicator(self):
        if self.recording_circle is None:
            self.recording_circle = self.canvas.create_oval(25, 25, 75, 75, fill="red")
        if self.recording_label is None:
            self.recording_label = self.canvas.create_text(50,90 , text="ON AIR!", font=("Cascadia Code", 12), fill="red")

    def hide_recording_indicator(self):
        if self.recording_circle is not None:
            self.canvas.delete(self.recording_circle)
            self.recording_circle = None
        if self.recording_label is not None:
            self.canvas.delete(self.recording_label)
            self.recording_label = None

    def start_data_collection(self):
        for serial_port, ser in self.serial_ports.items():
            text_widget = self.port_widgets[serial_port]['text_widget']
            self.data_to_save[serial_port] = []
            t = threading.Thread(target=self.read_from_port, args=(ser, serial_port, text_widget))
            t.start()
            self.threads.append(t)

    def read_from_port(self, ser, serial_port, text_widget):
        try:
            spreadsheet = self.gspread_client.open_by_key(self.spreadsheet_id.get())
            sheet = self.get_or_create_worksheet(spreadsheet, serial_port)

            cached_data = []  # List to store data when internet is down
            send_interval = 5  # Time interval to attempt sending cached data (in seconds)
            last_send_time = time.time()

            while not stop_event.is_set():
                try:
                    data = ser.readline().decode('utf-8', errors='replace').strip()
                except Exception as e:
                    print(f"Exception reading from serial port: {e}")
                    break  # Exit the loop
                if not data:
                    continue  # Skip if no data
                data_list = data.split(",")
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                # Check for "PelletInWell" and ignore
                if "PelletInWell" in data:
                    pass  # Do nothing for PelletInWell
                elif "Pellet" in data:
                    # Update last event time and start recording if necessary
                    with self.recording_locks[serial_port]:
                        self.last_event_times[serial_port] = datetime.datetime.now()
                        if not self.recording_states[serial_port]:
                            self.recording_states[serial_port] = True
                            threading.Thread(target=self.record_video, args=(serial_port,)).start()
                data_list = data_list[1:]  # Remove initial empty element if any
                if len(data_list) == len(COLUMN_HEADERS) - 1:
                    row_data = [timestamp] + data_list
                    cached_data.append(row_data)
                    text_widget.insert(tk.END, f"Data logged: {data_list}\n")
                    text_widget.see(tk.END)
                    self.data_to_save[serial_port].append(row_data)
                else:
                    print(f"Warning: Data length {len(data_list)} does not match header length {len(COLUMN_HEADERS) - 1}")

                # Periodically attempt to send cached data
                current_time = time.time()
                if cached_data and (current_time - last_send_time >= send_interval):
                    try:
                        # Append all cached data at once for efficiency
                        sheet.append_rows(cached_data)
                        print(f"Cached data sent to Google Sheets for {serial_port}")
                        cached_data.clear()
                    except gspread.exceptions.APIError as e:
                        error_code = e.response.status_code
                        print(f"Failed to send cached data for {serial_port}: {e}")
                        if error_code == 429:
                            print("Quota exceeded, backing off...")
                            time.sleep(60)  # Wait before retrying
                    except Exception as e:
                        print(f"Failed to send cached data for {serial_port}: {e}")
                    last_send_time = current_time  # Update last send time
                time.sleep(0.1)  # Small delay to avoid busy waiting
        except Exception as e:
            print(f"Error in read_from_port: {e}")
        finally:
            print(f"Thread for {serial_port} exiting")

    def record_video(self, serial_port):
        if serial_port not in self.cameras:
            print(f"No camera associated with serial port {serial_port}")
            return
        cam = self.cameras[serial_port]
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        port_identifier = self.port_widgets[serial_port]['frame'].cget('text')
        path = os.path.join(self.save_path, self.experimenter_name.get(), self.experiment_name.get(), port_identifier)
        os.makedirs(path, exist_ok=True)
        filename = os.path.join(path, f"{port_identifier}_camera_{timestamp}.avi")

        frame_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
        if frame_width == 0 or frame_height == 0:
            print(f"Camera at index {self.device_mapping[serial_port]} is not returning frames.")
            with self.recording_locks[serial_port]:
                self.recording_states[serial_port] = False
                self.last_event_times[serial_port] = None
            return
        out = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*'XVID'), 20.0, (frame_width, frame_height))

        try:
            while True:
                with self.recording_locks[serial_port]:
                    last_event_time = self.last_event_times[serial_port]
                if last_event_time is None:
                    break
                time_since_last_event = (datetime.datetime.now() - last_event_time).total_seconds()
                if time_since_last_event > 30:
                    break  # No 'Pellet' event within the last 30 seconds, stop recording
                # Else, continue recording
                ret, frame = cam.read()
                if ret:
                    out.write(frame)
                else:
                    print(f"Failed to read frame from camera at index {self.device_mapping[serial_port]}")
                    break
                time.sleep(0.05)  # Adjust as needed for frame rate
        finally:
            out.release()
            with self.recording_locks[serial_port]:
                self.recording_states[serial_port] = False
                self.last_event_times[serial_port] = None
            print(f"Video saved as {filename}")

    def get_or_create_worksheet(self, spreadsheet, title):
        try:
            return spreadsheet.worksheet(title)
        except gspread.exceptions.WorksheetNotFound:
            sheet = spreadsheet.add_worksheet(title=title, rows="1000", cols="20")
            sheet.append_row(COLUMN_HEADERS)
            return sheet

    def stop_logging(self):
        stop_event.set()
        # Wait for threads to finish
        for t in self.threads:
            t.join()
        self.hide_recording_indicator()
        self.save_all_data()
        # Now it's safe to close serial ports and release cameras
        for ser in self.serial_ports.values():
            ser.close()
        for cam in self.cameras.values():
            cam.release()
        self.root.quit()
        self.root.destroy()
        print("Monitoring stopped and data saved.")

    def save_all_data(self):
        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        folder = os.path.join(self.save_path, self.experimenter_name.get(), f"{self.experiment_name.get()}_{current_time}")
        os.makedirs(folder, exist_ok=True)
        for serial_port, data_rows in self.data_to_save.items():
            port_identifier = self.port_widgets[serial_port]['frame'].cget('text')
            filename_user = f"{folder}/{port_identifier}_{current_time}.csv"
            with open(filename_user, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(COLUMN_HEADERS)
                writer.writerows(data_rows)

# Main execution
if __name__ == "__main__":
    splash_root = tk.Tk()
    splash_screen = SplashScreen(splash_root)
    splash_root.mainloop()

    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()


In [10]:
a= 2
b = 4
c = 2*4
d= "DeepLabCut "
f= True

print (d*4)
print (c/a)

SyntaxError: invalid syntax (1675018806.py, line 9)

In [11]:
if
a==2,
    b==4
    print("YES")

SyntaxError: invalid syntax (72440094.py, line 1)